In [1]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import google.generativeai as genai
pd.set_option('display.max_columns', None)

In [3]:
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

# Get the API key from environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [4]:
employee_data = pd.read_csv("ds_salaries.csv")
employee_data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Save the dataframe to an SQLite database

In [5]:
connection = sqlite3.connect("salaries.db")
cursor = connection.cursor()

# Check if the table already exists
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='salaries';")
table_exists = cursor.fetchone()

if not table_exists:
    # Write the DataFrame to the SQL table
    employee_data.to_sql(name="salaries", con=connection, if_exists='fail')
else:
    print("Table 'salaries' already exists. Skipping the write operation.")

# Close the connection
connection.close()

Table 'salaries' already exists. Skipping the write operation.


## Setup the Logger

In [6]:
# @dataclass
# class Event:
#     event: str
#     timestamp: str
#     text: str


# def _current_time() -> str:
#     return datetime.now(timezone.utc).isoformat()


# class LLMCallbackHandler(BaseCallbackHandler):
#     def __init__(self, log_path: Path):
#         self.log_path = log_path

#     def on_llm_start(
#         self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
#     ) -> Any:
#         """Run when LLM starts running."""
#         assert len(prompts) == 1
#         event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
#         with self.log_path.open("a", encoding="utf-8") as file:
#             file.write(json.dumps(asdict(event)) + "\n")

#     def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
#         """Run when LLM ends running."""
#         generation = response.generations[-1][-1].message.content
#         event = Event(event="llm_end", timestamp=_current_time(), text=generation)
#         with self.log_path.open("a", encoding="utf-8") as file:
#             file.write(json.dumps(asdict(event)) + "\n")

## Setup the LLM

In [7]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     openai_api_base="https://api.groq.com/openai/v1",
#     openai_api_key=groq_api_key,
#     model="llama3-70b-8192",
#     temperature=0
# )

#from langchain_google_genai import ChatGoogleGenerativeAI

#load_dotenv()

# call gemini model
# llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',
#                             verbose=True,
#                             temperature=0.5,
#                             goggle_api_key=os.getenv('GOOGLE_API_KEY'))

# from crewai import LLM
# groq_llm = LLM(
#     model="groq/llama3-8b-8192",
#     temperature=0,
#     api_key=groq_api_key,
#     base_url="https://api.groq.com/openai/v1"
# )

from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama3-8b-8192",
    temperature=0.1,
    api_key=groq_api_key
)

In [8]:
#Sample test Invoking
# result = llm.invoke("What is Large language Modelling?")
# result.content

## Establisig a Database Connection

In [9]:
db = SQLDatabase.from_uri("sqlite:///salaries.db")

## Tool 1: List all the tables in the Database

In [10]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

Using Tool: list_tables


'salaries'

## Tool 2 : Return the schema and sample rows for a given list of tables

In [11]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

print(tables_schema.run("salaries"))

Using Tool: tables_schema

CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2023	SE	FT	Principal Data Scientist	80000	EUR	85847	ES	100	ES	L
1	2023	MI	CT	ML Engineer	30000	USD	30000	US	100	US	S
2	2023	MI	CT	ML Engineer	25500	USD	25500	US	100	US	S
*/


## Tool 3: Executes a given SQL query

In [12]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")

Using Tool: execute_sql


"[(0, 2023, 'SE', 'FT', 'Principal Data Scientist', 80000, 'EUR', 85847, 'ES', 100, 'ES', 'L'), (1, 2023, 'MI', 'CT', 'ML Engineer', 30000, 'USD', 30000, 'US', 100, 'US', 'S'), (2, 2023, 'MI', 'CT', 'ML Engineer', 25500, 'USD', 25500, 'US', 100, 'US', 'S'), (3, 2023, 'SE', 'FT', 'Data Scientist', 175000, 'USD', 175000, 'CA', 100, 'CA', 'M'), (4, 2023, 'SE', 'FT', 'Data Scientist', 120000, 'USD', 120000, 'CA', 100, 'CA', 'M')]"

## Tool 4 : checks the SQL query before executing it

In [13]:
# @tool("check_sql")
# def check_sql(sql_query: str) -> str:
#     """
#     Use this tool to double check if your query is correct before executing it. Always use this
#     tool before executing a query with `execute_sql`.
#     """
#     return QuerySQLCheckerTool(db=db, llm=groq_llm).invoke({"query": sql_query})

# check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

# Create Agents
## 
Agent 1 : Database Developer Agent will construct and execute SQL queries

In [14]:
#list_tables

In [15]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
    """
    ),
    llm=llm,
    verbose=True,
    tools=[list_tables, tables_schema, execute_sql],
    allow_delegation=False,
)

## Agent 2 : Data Analyst Agent will analyze the database data response and write a detailed response

In [16]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

Create Tasks

Task 1 : extract the data required for the user query

In [17]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)


In [18]:
# crew = Crew(
#     agents=[sql_dev, data_analyst, report_writer],
#     tasks=[extract_data, analyze_data, write_report],
#     process=Process.sequential,
#     verbose=True  # or verbose=False
# )

crew = Crew(
    agents=[sql_dev, data_analyst],
    tasks=[extract_data, analyze_data],
    process=Process.sequential,
    verbose=True,
    memory=False,
    output_log_file="crew.log",
)

In [19]:
inputs = {
    "query": "Effects on salary (in USD) based on company location, size and employee experience"
}
result = crew.kickoff(inputs=inputs)

 [2025-01-21 19:35:41][DEBUG]: == Working Agent: Senior Database Developer
 [2025-01-21 19:35:41][INFO]: == Starting Task: Extract data that is required for the query Effects on salary (in USD) based on company location, size and employee experience.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by listing the available tables in the database to understand what data I have to work with.

Action: list_tables

Action Input: {}
 

salaries

Thought: Now that I have the list of available tables, I need to understand the schema of the tables to determine which columns I can use for my query.

Action: tables_schema

Action Input: { "tables": "salaries" }
 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salar

In [20]:
print(result)

The provided dataset contains information on company location, size, and employee experience, along with their corresponding salaries in USD. To analyze the effects of these factors on salary, I will first examine the distribution of salaries across different company locations, sizes, and employee experiences.

**Company Location:**

The dataset contains information on company locations, which can be categorized into three regions: AE, AL, and SE. The average salary for each region is as follows:

| Region | Average Salary (USD) |
| --- | --- |
| AE | 93,750.00 |
| AL | 25,000.00 |
| SE | 87,500.00 |

From this analysis, it is clear that the average salary varies significantly across different regions. The AE region has the highest average salary, while the AL region has the lowest. This suggests that company location has a significant impact on salary.

**Company Size:**

The dataset also contains information on company size, which can be categorized into two categories: Large (L) and

In [23]:
#data_results = extract_data.execute()

In [25]:
#print(data_results)